# 0. **Install PySpark**

In [3]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=639a7efe57abc3a225f289bf6ff8721765883f739e7e285c1689caa513c9279b
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


# 1. **Initialize Spark session**:


In [6]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('SparkByExamples.com').getOrCreate()

   Initializes a Spark session with the application name 'SparkByExamples.com'.


# 2. **Create DataFrame**:


In [7]:
data = [('James','Smith','M',3000), ('Anna','Rose','F',4100), ('Robert','Williams','M',6200)]
columns = ["firstname", "lastname", "gender", "salary"]
df = spark.createDataFrame(data=data, schema=columns)

df.show()

+---------+--------+------+------+
|firstname|lastname|gender|salary|
+---------+--------+------+------+
|    James|   Smith|     M|  3000|
|     Anna|    Rose|     F|  4100|
|   Robert|Williams|     M|  6200|
+---------+--------+------+------+



Creates a DataFrame from the sample data and schema, then displays it.


# 3. **Check if a column exists**:


In [8]:
if 'salary1' not in df.columns:
    print("Column 'salary1' does not exist.")

Column 'salary1' does not exist.


   Checks if the column 'salary1' exists in the DataFrame columns.


# 4. **Add a constant column**:


In [9]:
from pyspark.sql.functions import lit
df.withColumn("bonus_percent", lit(0.3)).show()

+---------+--------+------+------+-------------+
|firstname|lastname|gender|salary|bonus_percent|
+---------+--------+------+------+-------------+
|    James|   Smith|     M|  3000|          0.3|
|     Anna|    Rose|     F|  4100|          0.3|
|   Robert|Williams|     M|  6200|          0.3|
+---------+--------+------+------+-------------+



   Adds a new column 'bonus_percent' with a constant value of 0.3.



`lit` is a function from the pyspark.sql.functions module that is used to create a column with a literal (constant) value. This can be useful when you want to add a new column to a DataFrame with a constant value or when you need to use a constant value in an expression.

# 5. **Add a column from an existing column**:


In [10]:
df.withColumn("bonus_amount", df.salary * 0.3).show()

+---------+--------+------+------+------------+
|firstname|lastname|gender|salary|bonus_amount|
+---------+--------+------+------+------------+
|    James|   Smith|     M|  3000|       900.0|
|     Anna|    Rose|     F|  4100|      1230.0|
|   Robert|Williams|     M|  6200|      1860.0|
+---------+--------+------+------+------------+



   Adds a new column 'bonus_amount' by multiplying the 'salary' column by 0.3.


# 6. **Add a column by concatenating existing columns**:


In [16]:
from pyspark.sql.functions import concat_ws
df.withColumn("name", concat_ws(" ", "firstname", "lastname")).show()

+---------+--------+------+------+---------------+
|firstname|lastname|gender|salary|           name|
+---------+--------+------+------+---------------+
|    James|   Smith|     M|  3000|    James Smith|
|     Anna|    Rose|     F|  4100|      Anna Rose|
|   Robert|Williams|     M|  6200|Robert Williams|
+---------+--------+------+------+---------------+



   Adds a new column 'name' by concatenating 'firstname' and 'lastname' columns.


The `concat_ws` function in PySpark is used to concatenate multiple columns into a single column, with a specified separator between each value. The "ws" stands for "with separator". This function is particularly useful when you need to combine multiple columns into one string column with a specific delimiter.

# 7. **Add the current date column**:


In [12]:
from pyspark.sql.functions import current_date
df.withColumn("current_date", current_date()).show()

+---------+--------+------+------+------------+
|firstname|lastname|gender|salary|current_date|
+---------+--------+------+------+------------+
|    James|   Smith|     M|  3000|  2024-07-15|
|     Anna|    Rose|     F|  4100|  2024-07-15|
|   Robert|Williams|     M|  6200|  2024-07-15|
+---------+--------+------+------+------------+



   Adds a new column 'current_date' with the current date.


The `current_date` function in PySpark is used to retrieve the current date as a DateType column. This function is particularly useful when you need to add the current date to a DataFrame or use the current date in date-based calculations and comparisons.

# 8. **Add a column based on condition**:


In [13]:
from pyspark.sql.functions import when
df.withColumn("grade",
    when((df.salary < 4000), lit("A"))
    .when((df.salary >= 4000) & (df.salary <= 5000), lit("B"))
    .otherwise(lit("C"))
).show()

+---------+--------+------+------+-----+
|firstname|lastname|gender|salary|grade|
+---------+--------+------+------+-----+
|    James|   Smith|     M|  3000|    A|
|     Anna|    Rose|     F|  4100|    B|
|   Robert|Williams|     M|  6200|    C|
+---------+--------+------+------+-----+



   Adds a new column 'grade' based on the salary value:
   - 'A' if salary is less than 4000.
   - 'B' if salary is between 4000 and 5000.
   - 'C' otherwise.


The `when` function in PySpark is used to apply conditional logic to DataFrame columns. It's similar to the CASE WHEN statement in SQL. The when function allows you to create a new column based on conditions you define, and you can chain multiple when calls together to handle multiple conditions. If none of the conditions are met, you can use the otherwise method to provide a default value.

# 9. **Add columns using select**:


In [14]:
df.select("firstname", "salary", lit(0.3).alias("bonus")).show()
df.select("firstname", "salary", (df.salary * 0.3).alias("bonus_amount")).show()
df.select("firstname", "salary", current_date().alias("today_date")).show()

+---------+------+-----+
|firstname|salary|bonus|
+---------+------+-----+
|    James|  3000|  0.3|
|     Anna|  4100|  0.3|
|   Robert|  6200|  0.3|
+---------+------+-----+

+---------+------+------------+
|firstname|salary|bonus_amount|
+---------+------+------------+
|    James|  3000|       900.0|
|     Anna|  4100|      1230.0|
|   Robert|  6200|      1860.0|
+---------+------+------------+

+---------+------+----------+
|firstname|salary|today_date|
+---------+------+----------+
|    James|  3000|2024-07-15|
|     Anna|  4100|2024-07-15|
|   Robert|  6200|2024-07-15|
+---------+------+----------+



   Adds new columns 'bonus', 'bonus_amount', and 'today_date' using the `select` method.


# 10. **Add columns using SQL**:


In [15]:
df.createOrReplaceTempView("PER")

spark.sql("SELECT firstname, salary, '0.3' AS bonus FROM PER").show()
spark.sql("SELECT firstname, salary, salary * 0.3 AS bonus_amount FROM PER").show()
spark.sql("SELECT firstname, salary, current_date() AS today_date FROM PER").show()

spark.sql(
    "SELECT firstname, salary, " +
    "CASE WHEN salary < 4000 THEN 'A' " +
    "WHEN salary >= 4000 AND salary <= 5000 THEN 'B' " +
    "ELSE 'C' END AS grade " +
    "FROM PER"
).show()

+---------+------+-----+
|firstname|salary|bonus|
+---------+------+-----+
|    James|  3000|  0.3|
|     Anna|  4100|  0.3|
|   Robert|  6200|  0.3|
+---------+------+-----+

+---------+------+------------+
|firstname|salary|bonus_amount|
+---------+------+------------+
|    James|  3000|       900.0|
|     Anna|  4100|      1230.0|
|   Robert|  6200|      1860.0|
+---------+------+------------+

+---------+------+----------+
|firstname|salary|today_date|
+---------+------+----------+
|    James|  3000|2024-07-15|
|     Anna|  4100|2024-07-15|
|   Robert|  6200|2024-07-15|
+---------+------+----------+

+---------+------+-----+
|firstname|salary|grade|
+---------+------+-----+
|    James|  3000|    A|
|     Anna|  4100|    B|
|   Robert|  6200|    C|
+---------+------+-----+



Uses SQL to add new columns 'bonus', 'bonus_amount', 'today_date', and 'grade' based on conditions.


The `createOrReplaceTempView` method in PySpark is used to create a temporary view from a DataFrame. This temporary view can be queried using SQL syntax within the same Spark session. This method is particularly useful when you want to perform SQL-like operations on a DataFrame.

**Usage of `createOrReplaceTempView`**

1. **Syntax**:
   ```python
   df.createOrReplaceTempView("view_name")
   ```
   - `df`: The DataFrame you want to create the view from.
   - `"view_name"`: The name of the temporary view.

2. **Purpose**:
   - Allows you to query the DataFrame using SQL.
   - Useful for performing complex SQL operations and aggregations.
   - Integrates with other Spark SQL functionalities.

**Example**

Here's an example demonstrating how to use `createOrReplaceTempView` and perform SQL queries on a DataFrame:
